# Parsing ASVS JSON into Pandas DataFrame

This notebook will read the `asvs.json` file and parse all the Bible verses into a pandas DataFrame for easy analysis and manipulation.


In [1]:
# Import the necessary libraries
# json: for reading and parsing JSON files
# pandas: for creating and working with DataFrames (tables of data)
import json
import pandas as pd


In [2]:
# Open the asvs.json file in read mode
# 'r' means read mode, and 'utf-8' ensures we can read special characters properly
# The 'with' statement automatically closes the file when we're done
with open('asvs.json', 'r', encoding='utf-8') as file:
    # Load the JSON data from the file
    # json.load() reads the entire JSON structure and converts it to Python data structures
    data = json.load(file)


In [3]:
# Check what type of data structure we have
# This helps us understand how to work with the data
print(f"Type of data: {type(data)}")

# If it's a dictionary, check what keys it has
# If it's a list, check how many items
if isinstance(data, dict):
    print(f"Keys in dictionary: {list(data.keys())}")
    # If there's a key that contains the verses, let's see its structure
    if 'verses' in data or 'verse' in data or 'data' in data:
        key = 'verses' if 'verses' in data else ('verse' if 'verse' in data else 'data')
        print(f"Number of items in '{key}': {len(data[key])}")
        print(f"First item structure: {list(data[key][0].keys()) if data[key] else 'Empty'}")
elif isinstance(data, list):
    print(f"Number of verses: {len(data)}")
    if len(data) > 0:
        print(f"First verse keys: {list(data[0].keys())}")
        print(f"First verse example: {data[0]}")


Type of data: <class 'dict'>
Keys in dictionary: ['metadata', 'verses']
Number of items in 'verses': 31086
First item structure: ['book_name', 'book', 'chapter', 'verse', 'text']


In [4]:
# Extract the verses list from the data
# The JSON might be a dictionary with a 'verses' key, or it might be a list directly
if isinstance(data, dict):
    # If it's a dictionary, find the key that contains the list of verses
    # Common keys might be 'verses', 'verse', 'data', or we'll take the first list value
    verses_list = None
    for key in data.keys():
        if isinstance(data[key], list):
            verses_list = data[key]
            print(f"Found verses in key: '{key}'")
            break
    if verses_list is None:
        print("Error: Could not find a list of verses in the dictionary")
        verses_list = []
else:
    # If data is already a list, use it directly
    verses_list = data

# Print how many verses we found
print(f"Total number of verses: {len(verses_list)}")


Found verses in key: 'verses'
Total number of verses: 31086


In [5]:
# Convert the list of verse dictionaries into a pandas DataFrame
# pd.DataFrame() takes a list of dictionaries and creates a table where:
# - Each dictionary becomes a row
# - Each key in the dictionaries becomes a column
df = pd.DataFrame(verses_list)

# Display the first few rows to see what we have
# .head() shows the first 5 rows by default
print("First 5 rows of the DataFrame:")
print(df.head())

# Display information about the DataFrame
# .info() shows column names, data types, and how many non-null values
print("\nDataFrame Info:")
print(df.info())


First 5 rows of the DataFrame:
  book_name  book  chapter  verse  \
0   Genesis     1        1      1   
1   Genesis     1        1      2   
2   Genesis     1        1      3   
3   Genesis     1        1      4   
4   Genesis     1        1      5   

                                                text  
0  In the beginning{H7225} God{H430} created{H125...  
1  And the earth{H776} was{H1961} waste{H8414} an...  
2  And God{H430} said,{H559} Let there be{H1961} ...  
3  And God{H430} saw{H7200} the light,{H216} that...  
4  And God{H430} called{H7121} the light{H216} Da...  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31086 entries, 0 to 31085
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   book_name  31086 non-null  object
 1   book       31086 non-null  int64 
 2   chapter    31086 non-null  int64 
 3   verse      31086 non-null  int64 
 4   text       31086 non-null  object
dtypes: int64(3), 

In [6]:
# Display basic statistics about the DataFrame
# .describe() gives us summary statistics for numeric columns
print("Summary Statistics:")
print(df.describe())

# Show the column names
print(f"\nColumn names: {list(df.columns)}")

# Show the shape (number of rows and columns)
print(f"\nDataFrame shape: {df.shape} (rows, columns)")


Summary Statistics:
               book       chapter         verse
count  31086.000000  31086.000000  31086.000000
mean      22.486071     20.633983     17.039471
std       16.494070     23.413676     14.075116
min        1.000000      1.000000      1.000000
25%        9.000000      6.000000      7.000000
50%       19.000000     14.000000     14.000000
75%       40.000000     26.000000     23.000000
max       66.000000    150.000000    176.000000

Column names: ['book_name', 'book', 'chapter', 'verse', 'text']

DataFrame shape: (31086, 5) (rows, columns)


In [7]:
# Display a sample verse to see the structure
# .iloc[0] gets the first row (index 0)
print("Example verse (first row):")
print(df.iloc[0])


Example verse (first row):
book_name                                              Genesis
book                                                         1
chapter                                                      1
verse                                                        1
text         In the beginning{H7225} God{H430} created{H125...
Name: 0, dtype: object


## Extracting Strong's Numbers

Now we'll extract all the Strong's numbers (the codes inside curly braces like {H7225} or {G2532}) from the text column and store them in a new column called "strongs".


In [8]:
# Import the 're' module for regular expressions
# Regular expressions (regex) are patterns used to find and extract specific text patterns
# We'll use it to find all Strong's numbers in the format {H1234} or {G1234}
import re


In [9]:
# Define a function to extract Strong's numbers from a text string
# This function will be applied to each row of the DataFrame
def extract_strongs(text):
    """
    Extract all Strong's numbers from a text string.
    
    Parameters:
    text (str): The verse text containing Strong's numbers in format {H1234} or {G1234}
    
    Returns:
    list: A list of all Strong's numbers found in the text
    """
    # Use regex to find all patterns that match {letter followed by numbers}
    # Pattern explanation:
    #   \{        - matches the opening curly brace {
    #   ([HG]\d+) - matches H or G followed by one or more digits (captured in group 1)
    #   \}        - matches the closing curly brace }
    # re.findall() returns a list of all matches (the captured group, which is the Strong's number)
    strongs_list = re.findall(r'\{([HG]\d+)\}', text)
    
    # Return the list of Strong's numbers
    return strongs_list


In [10]:
# Apply the extract_strongs function to each row in the 'text' column
# .apply() runs the function on each value in the column
# This creates a new column called 'strongs' containing lists of Strong's numbers
df['strongs'] = df['text'].apply(extract_strongs)

# Display the first few rows to see the new column
print("First 5 rows with the new 'strongs' column:")
print(df[['book_name', 'chapter', 'verse', 'strongs']].head())

# Show an example of the text and extracted Strong's numbers side by side
print("\n" + "="*80)
print("Example: Text vs Extracted Strong's Numbers")
print("="*80)
for i in range(3):
    print(f"\nRow {i}:")
    print(f"  Text: {df.iloc[i]['text'][:100]}...")  # Show first 100 characters
    print(f"  Strong's: {df.iloc[i]['strongs']}")


First 5 rows with the new 'strongs' column:
  book_name  chapter  verse                                            strongs
0   Genesis        1      1            [H7225, H430, H1254, H8064, H853, H776]
1   Genesis        1      2  [H776, H1961, H8414, H922, H2822, H6440, H8415...
2   Genesis        1      3                    [H430, H559, H1961, H216, H216]
3   Genesis        1      4  [H430, H7200, H216, H3588, H2896, H430, H914, ...
4   Genesis        1      5  [H430, H7121, H216, H3117, H2822, H7121, H3915...

Example: Text vs Extracted Strong's Numbers

Row 0:
  Text: In the beginning{H7225} God{H430} created{H1254} the heavens{H8064} and{H853} the earth.{H776}...
  Strong's: ['H7225', 'H430', 'H1254', 'H8064', 'H853', 'H776']

Row 1:
  Text: And the earth{H776} was{H1961} waste{H8414} and void;{H922} and darkness{H2822} was upon the face{H6...
  Strong's: ['H776', 'H1961', 'H8414', 'H922', 'H2822', 'H6440', 'H8415', 'H7307', 'H430', 'H7363', 'H5921', 'H6440', 'H4325']

Row 2:
  Te

In [11]:
# Check the DataFrame info again to see the new column
print("Updated DataFrame Info:")
print(df.info())

# Show some statistics about the Strong's numbers
# Count how many Strong's numbers are in each verse
df['strongs_count'] = df['strongs'].apply(len)

print(f"\nStatistics about Strong's numbers per verse:")
print(f"  Average number of Strong's per verse: {df['strongs_count'].mean():.2f}")
print(f"  Minimum: {df['strongs_count'].min()}")
print(f"  Maximum: {df['strongs_count'].max()}")
print(f"  Total unique Strong's numbers: {len(set([item for sublist in df['strongs'] for item in sublist]))}")


Updated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31086 entries, 0 to 31085
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   book_name  31086 non-null  object
 1   book       31086 non-null  int64 
 2   chapter    31086 non-null  int64 
 3   verse      31086 non-null  int64 
 4   text       31086 non-null  object
 5   strongs    31086 non-null  object
dtypes: int64(3), object(3)
memory usage: 1.4+ MB
None

Statistics about Strong's numbers per verse:
  Average number of Strong's per verse: 11.21
  Minimum: 1
  Maximum: 49
  Total unique Strong's numbers: 14022


## Downloading Strong's Dictionary from JavaScript Files

We'll download and parse the `strongs-greek-dictionary.js` and `strongs-hebrew-dictionary.js` files from the OpenScriptures repository. These JavaScript files contain the dictionary data in a structured format.


In [12]:
# Import additional libraries needed for downloading and parsing
# requests: for downloading files from the internet
# os: for file operations (checking if files exist, creating directories)
# re: for regular expressions to parse JavaScript files
import requests
import os
import re


In [13]:
# Define the base URL for the GitHub repository
# We'll download the JavaScript dictionary files
base_url = "https://raw.githubusercontent.com/openscriptures/strongs/master"

# URLs for the Greek and Hebrew dictionary JavaScript files
greek_js_url = f"{base_url}/strongs-greek-dictionary.js"
hebrew_js_url = f"{base_url}/strongs-hebrew-dictionary.js"

# Local filenames where we'll save the downloaded files
greek_js_filename = "strongs-greek-dictionary.js"
hebrew_js_filename = "strongs-hebrew-dictionary.js"

print("Strong's Dictionary JavaScript Files:")
print(f"  Greek: {greek_js_url}")
print(f"  Hebrew: {hebrew_js_url}")


Strong's Dictionary JavaScript Files:
  Greek: https://raw.githubusercontent.com/openscriptures/strongs/master/strongs-greek-dictionary.js
  Hebrew: https://raw.githubusercontent.com/openscriptures/strongs/master/strongs-hebrew-dictionary.js


In [14]:
# Function to download a JavaScript dictionary file
def download_js_file(url, filename):
    """
    Download a JavaScript dictionary file if it doesn't already exist.
    
    Parameters:
    url: URL of the file to download
    filename: Local filename to save the file
    
    Returns:
    bool: True if download was successful or file already exists
    """
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        try:
            # Use requests.get() to download the file
            response = requests.get(url, stream=True, timeout=30)
            
            # Check if the download was successful (status code 200 means success)
            if response.status_code == 200:
                # Open a local file in write-binary mode ('wb') to save the downloaded content
                with open(filename, 'wb') as f:
                    # Write the content to the file in chunks (more efficient for large files)
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"✓ Successfully downloaded {filename}")
                return True
            else:
                print(f"✗ Error downloading {filename}. Status code: {response.status_code}")
                return False
        except Exception as e:
            print(f"✗ Error downloading {filename}: {e}")
            return False
    else:
        print(f"✓ File {filename} already exists, skipping download")
        return True

# Download both dictionary files
print("="*80)
greek_downloaded = download_js_file(greek_js_url, greek_js_filename)
print()
hebrew_downloaded = download_js_file(hebrew_js_url, hebrew_js_filename)
print("="*80)


✓ File strongs-greek-dictionary.js already exists, skipping download

✓ File strongs-hebrew-dictionary.js already exists, skipping download


In [15]:
# Read the Greek dictionary JavaScript file
# JavaScript files typically contain a variable assignment like: var strongsGreek = {...}
# We need to extract the JSON-like object from the JavaScript code
print("Reading Greek dictionary file...")
with open(greek_js_filename, 'r', encoding='utf-8') as f:
    greek_js_content = f.read()

print(f"Greek file size: {len(greek_js_content):,} characters")
print(f"First 500 characters:")
print(greek_js_content[:500])


Reading Greek dictionary file...
Greek file size: 1,111,565 characters
First 500 characters:
/**
 *                      Dictionary of Greek Words
 *                              taken from
 *                    Strong's Exhaustive Concordance
 *                                   by
 *                      James Strong, S.T.D., LL.D.
 *                                  1890
 * 
 * JSON version
 * ============
 * Copyright 2009, Open Scriptures. CC-BY-SA. Derived from XML.
 * $Id$
 *
 * XML e-text version
 * ==================
 * 
 * The XML version of this work was prepared in 2006 by U


In [16]:
# Function to extract JSON data from JavaScript file
# JavaScript files often have patterns like: var variableName = {...} or const variableName = {...}
def extract_json_from_js(js_content, variable_name):
    """
    Extract JSON data from a JavaScript file.
    
    Parameters:
    js_content: The JavaScript file content as a string
    variable_name: The name of the variable containing the dictionary (e.g., 'strongsGreek')
    
    Returns:
    dict: The extracted dictionary data
    """
    # Pattern to find the variable assignment
    # Matches: var/const/let variableName = { ... };
    # We'll look for the opening brace after the variable name and extract everything until the matching closing brace
    pattern = rf'(?:var|const|let)\s+{re.escape(variable_name)}\s*=\s*(\{{.*?\}});'
    
    # Try to find the pattern
    match = re.search(pattern, js_content, re.DOTALL)
    
    if match:
        # Extract the JSON-like string
        json_str = match.group(1)
        
        # Try to parse it as JSON
        try:
            return json.loads(json_str)
        except json.JSONDecodeError:
            # If it's not valid JSON, try to clean it up
            # JavaScript might have trailing commas or other issues
            # Remove trailing commas before closing braces/brackets
            json_str = re.sub(r',(\s*[}\]])', r'\1', json_str)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError as e:
                print(f"Error parsing JSON: {e}")
                return None
    else:
        # Alternative: look for the object directly without variable assignment
        # Sometimes the file might just be the object itself
        try:
            # Try to find the first { and last } and extract everything in between
            first_brace = js_content.find('{')
            last_brace = js_content.rfind('}')
            if first_brace != -1 and last_brace != -1 and last_brace > first_brace:
                json_str = js_content[first_brace:last_brace+1]
                # Clean up trailing commas
                json_str = re.sub(r',(\s*[}\]])', r'\1', json_str)
                return json.loads(json_str)
        except Exception as e:
            print(f"Alternative parsing failed: {e}")
    
    return None

# Try to extract the Greek dictionary
# Common variable names: strongsGreek, strongs_greek, greek, etc.
print("\nAttempting to extract Greek dictionary data...")
greek_dict = None

# Try different possible variable names
for var_name in ['strongsGreek', 'strongs_greek', 'greek', 'strongsGreekDictionary']:
    greek_dict = extract_json_from_js(greek_js_content, var_name)
    if greek_dict:
        print(f"✓ Found Greek dictionary with variable name: {var_name}")
        print(f"  Number of entries: {len(greek_dict)}")
        break

if not greek_dict:
    print("✗ Could not extract Greek dictionary. Let's examine the file structure...")
    # Show more of the file to understand its structure
    print("\nFirst 1000 characters of the file:")
    print(greek_js_content[:1000])



Attempting to extract Greek dictionary data...
✓ Found Greek dictionary with variable name: strongsGreek
  Number of entries: 5523


In [17]:
# Read the Hebrew dictionary JavaScript file
print("\n" + "="*80)
print("Reading Hebrew dictionary file...")
with open(hebrew_js_filename, 'r', encoding='utf-8') as f:
    hebrew_js_content = f.read()

print(f"Hebrew file size: {len(hebrew_js_content):,} characters")
print(f"First 500 characters:")
print(hebrew_js_content[:500])



Reading Hebrew dictionary file...
Hebrew file size: 1,870,416 characters
First 500 characters:
/**
 * A Concise Dictionary of the Words in the Hebrew Bible
 *    with their Renderings in the King James Version
 * by James Strong, LL.D., S.T.D.
 * 1894
 *
 * JSON version
 * ============
 * Copyright 2010, Open Scriptures. CC-BY-SA. Derived from XML.
 * $Id$
 *
 * XML e-text version
 * ==================
 * From a source by David Instone-Brewer (www.2LetterLookup.com):
 * "The data came from several PD sources which were full of errors. 
 * I cleaned them by comparing them to each other and


In [18]:
# Extract Hebrew dictionary data
print("\nAttempting to extract Hebrew dictionary data...")
hebrew_dict = None

# Try different possible variable names
for var_name in ['strongsHebrew', 'strongs_hebrew', 'hebrew', 'strongsHebrewDictionary']:
    hebrew_dict = extract_json_from_js(hebrew_js_content, var_name)
    if hebrew_dict:
        print(f"✓ Found Hebrew dictionary with variable name: {var_name}")
        print(f"  Number of entries: {len(hebrew_dict)}")
        break

if not hebrew_dict:
    print("✗ Could not extract Hebrew dictionary. Let's examine the file structure...")
    # Show more of the file to understand its structure
    print("\nFirst 1000 characters of the file:")
    print(hebrew_js_content[:1000])



Attempting to extract Hebrew dictionary data...
✓ Found Hebrew dictionary with variable name: strongsHebrew
  Number of entries: 8674


In [19]:
# Combine Greek and Hebrew dictionaries and create DataFrames
# First, let's see what structure the dictionaries have
if greek_dict:
    print("Greek dictionary structure:")
    # Get the first key to see the structure
    first_key = list(greek_dict.keys())[0] if greek_dict else None
    if first_key:
        print(f"  Sample key: {first_key}")
        print(f"  Sample entry: {greek_dict[first_key]}")
        print(f"  Entry type: {type(greek_dict[first_key])}")
        if isinstance(greek_dict[first_key], dict):
            print(f"  Entry keys: {list(greek_dict[first_key].keys())}")

if hebrew_dict:
    print("\nHebrew dictionary structure:")
    first_key = list(hebrew_dict.keys())[0] if hebrew_dict else None
    if first_key:
        print(f"  Sample key: {first_key}")
        print(f"  Sample entry: {hebrew_dict[first_key]}")
        print(f"  Entry type: {type(hebrew_dict[first_key])}")
        if isinstance(hebrew_dict[first_key], dict):
            print(f"  Entry keys: {list(hebrew_dict[first_key].keys())}")


Greek dictionary structure:
  Sample key: G1615
  Sample entry: {'strongs_def': ' to complete fully', 'derivation': 'from G1537 (ἐκ) and G5055 (τελέω);', 'translit': 'ekteléō', 'lemma': 'ἐκτελέω', 'kjv_def': 'finish'}
  Entry type: <class 'dict'>
  Entry keys: ['strongs_def', 'derivation', 'translit', 'lemma', 'kjv_def']

Hebrew dictionary structure:
  Sample key: H1
  Sample entry: {'lemma': 'אָב', 'xlit': 'ʼâb', 'pron': 'awb', 'derivation': 'a primitive word;', 'strongs_def': 'father, in a literal and immediate, or figurative and remote application', 'kjv_def': "chief, (fore-) father(-less), [idiom] patrimony, principal. Compare names in 'Abi-'."}
  Entry type: <class 'dict'>
  Entry keys: ['lemma', 'xlit', 'pron', 'derivation', 'strongs_def', 'kjv_def']


In [20]:
# Convert dictionaries to DataFrames
# We'll create a list of dictionaries, one for each Strong's entry
strongs_entries = []

# Process Greek dictionary
if greek_dict:
    print("Processing Greek dictionary...")
    for strongs_num, entry_data in greek_dict.items():
        # Create a dictionary for this entry
        entry = {
            'number': strongs_num,
            'language': 'G'
        }
        
        # If entry_data is a dictionary, add its keys as columns
        if isinstance(entry_data, dict):
            entry.update(entry_data)
        else:
            # If it's a string or other type, store it in a 'definition' column
            entry['definition'] = str(entry_data)
        
        strongs_entries.append(entry)
    print(f"  Added {len(greek_dict)} Greek entries")

# Process Hebrew dictionary
if hebrew_dict:
    print("Processing Hebrew dictionary...")
    for strongs_num, entry_data in hebrew_dict.items():
        # Create a dictionary for this entry
        entry = {
            'number': strongs_num,
            'language': 'H'
        }
        
        # If entry_data is a dictionary, add its keys as columns
        if isinstance(entry_data, dict):
            entry.update(entry_data)
        else:
            # If it's a string or other type, store it in a 'definition' column
            entry['definition'] = str(entry_data)
        
        strongs_entries.append(entry)
    print(f"  Added {len(hebrew_dict)} Hebrew entries")

# Create DataFrame from the list of entries
if strongs_entries:
    strongs_df = pd.DataFrame(strongs_entries)
    print(f"\n✓ Created Strong's dictionary DataFrame")
    print(f"  Shape: {strongs_df.shape} (rows, columns)")
    print(f"  Columns: {list(strongs_df.columns)}")
    print(f"\nFirst few rows:")
    print(strongs_df.head())
else:
    print("✗ No entries to create DataFrame")
    strongs_df = pd.DataFrame()


Processing Greek dictionary...
  Added 5523 Greek entries
Processing Hebrew dictionary...
  Added 8674 Hebrew entries

✓ Created Strong's dictionary DataFrame
  Shape: (14197, 9) (rows, columns)
  Columns: ['number', 'language', 'strongs_def', 'derivation', 'translit', 'lemma', 'kjv_def', 'xlit', 'pron']

First few rows:
  number language                                        strongs_def  \
0  G1615        G                                  to complete fully   
1  G2274        G   to make worse, i.e. vanquish (literally or fi...   
2  G4533        G                               Salmon, an Israelite   
3  G5073        G                                          quadruple   
4  G3892        G                                      transgression   

                                          derivation    translit       lemma  \
0                 from G1537 (ἐκ) and G5055 (τελέω);     ekteléō     ἐκτελέω   
1                    from the same as G2276 (ἥττον);      hēttáō       ἡττάω   
2   

In [21]:
# Display some example Strong's definitions
if not strongs_df.empty:
    print("Sample Strong's Definitions:")
    print("="*80)
    for i in range(min(5, len(strongs_df))):
        row = strongs_df.iloc[i]
        print(f"\n{row['number']} ({row['language']}):")
        # Show all column values except number and language
        for col in strongs_df.columns:
            if col not in ['number', 'language']:
                value = str(row[col])
                if len(value) > 200:
                    value = value[:200] + "..."
                print(f"  {col}: {value}")


Sample Strong's Definitions:

G1615 (G):
  strongs_def:  to complete fully
  derivation: from G1537 (ἐκ) and G5055 (τελέω);
  translit: ekteléō
  lemma: ἐκτελέω
  kjv_def: finish
  xlit: nan
  pron: nan

G2274 (G):
  strongs_def:  to make worse, i.e. vanquish (literally or figuratively); by implication, to rate lower
  derivation: from the same as G2276 (ἥττον);
  translit: hēttáō
  lemma: ἡττάω
  kjv_def: be inferior, overcome
  xlit: nan
  pron: nan

G4533 (G):
  strongs_def:  Salmon, an Israelite
  derivation: of Hebrew origin (H08012);
  translit: Salmṓn
  lemma: Σαλμών
  kjv_def: Salmon
  xlit: nan
  pron: nan

G5073 (G):
  strongs_def:  quadruple
  derivation: from G5064 (τέσσαρες) and a derivative of the base of G4118 (πλεῖστος);
  translit: tetraplóos
  lemma: τετραπλόος
  kjv_def: fourfold
  xlit: nan
  pron: nan

G3892 (G):
  strongs_def:  transgression
  derivation: from the same as G3891 (παρανομέω);
  translit: paranomía
  lemma: παρανομία
  kjv_def: iniquity
  xlit: nan
 

In [22]:
# Save the Strong's dictionary DataFrame to files for future use
if not strongs_df.empty:
    # Save as CSV
    strongs_csv_filename = "strongs_dictionary.csv"
    strongs_df.to_csv(strongs_csv_filename, index=False, encoding='utf-8')
    print(f"✓ Saved Strong's dictionary to {strongs_csv_filename}")
    
    # Save as JSON for easy loading later
    strongs_json_filename = "strongs_dictionary.json"
    # Convert DataFrame to dictionary format for JSON
    strongs_dict_for_json = strongs_df.set_index('number').to_dict('index')
    with open(strongs_json_filename, 'w', encoding='utf-8') as f:
        json.dump(strongs_dict_for_json, f, ensure_ascii=False, indent=2)
    print(f"✓ Saved Strong's dictionary to {strongs_json_filename}")
    
    print(f"\n✓ Strong's dictionary DataFrame ready!")
    print(f"  - Total entries: {len(strongs_df)}")
    print(f"  - Columns: {list(strongs_df.columns)}")
    print(f"  - Use 'strongs_df' to access the data")
else:
    print("✗ No data to save")


✓ Saved Strong's dictionary to strongs_dictionary.csv
✓ Saved Strong's dictionary to strongs_dictionary.json

✓ Strong's dictionary DataFrame ready!
  - Total entries: 14197
  - Columns: ['number', 'language', 'strongs_def', 'derivation', 'translit', 'lemma', 'kjv_def', 'xlit', 'pron']
  - Use 'strongs_df' to access the data


## Creating Enhanced DataFrame with Strong's Definitions

Now we'll create an enhanced version of the DataFrame where Strong's numbers are replaced with their actual definitions from the Strong's dictionary.


In [23]:
# Create a copy of the original DataFrame
# .copy() creates a deep copy so changes to df_strongs_enhanced won't affect the original df
df_strongs_enhanced = df.copy()

print(f"Created df_strongs_enhanced with {len(df_strongs_enhanced)} rows")
print(f"Columns: {list(df_strongs_enhanced.columns)}")


Created df_strongs_enhanced with 31086 rows
Columns: ['book_name', 'book', 'chapter', 'verse', 'text', 'strongs', 'strongs_count']


In [24]:
# Create a mapping dictionary from Strong's numbers to their definitions
# This will allow us to quickly look up definitions by Strong's number
# We'll use the 'strongs_def' column from strongs_df as the definition

# Check if strongs_df exists and has the expected columns
if 'strongs_df' in globals() and not strongs_df.empty:
    # Create a dictionary mapping Strong's numbers to their definitions
    # The 'number' column contains the Strong's number (e.g., 'G2532', 'H430')
    # The 'strongs_def' column contains the definition
    
    # First, check what columns are available
    print("Available columns in strongs_df:")
    print(list(strongs_df.columns))
    
    # Create the mapping dictionary
    # Use 'strongs_def' if it exists, otherwise try other common column names
    definition_column = None
    for col in ['strongs_def', 'definition', 'kjv_def', 'lemma']:
        if col in strongs_df.columns:
            definition_column = col
            break
    
    if definition_column:
        # Create dictionary: {Strong's number: definition}
        strongs_to_def = dict(zip(strongs_df['number'], strongs_df[definition_column]))
        print(f"\n✓ Created mapping dictionary using '{definition_column}' column")
        print(f"  Total mappings: {len(strongs_to_def)}")
        print(f"\nSample mappings:")
        # Show first 5 mappings
        for i, (num, defn) in enumerate(list(strongs_to_def.items())[:5]):
            defn_preview = str(defn)[:50] + "..." if len(str(defn)) > 50 else str(defn)
            print(f"  {num}: {defn_preview}")
    else:
        print("✗ Could not find a definition column in strongs_df")
        strongs_to_def = {}
else:
    print("✗ strongs_df is not available. Please run the previous cells to load the Strong's dictionary.")
    strongs_to_def = {}


Available columns in strongs_df:
['number', 'language', 'strongs_def', 'derivation', 'translit', 'lemma', 'kjv_def', 'xlit', 'pron']

✓ Created mapping dictionary using 'strongs_def' column
  Total mappings: 14197

Sample mappings:
  G1615:  to complete fully
  G2274:  to make worse, i.e. vanquish (literally or figura...
  G4533:  Salmon, an Israelite
  G5073:  quadruple
  G3892:  transgression


In [25]:
# Function to replace Strong's numbers in text with their definitions
# The text contains patterns like {G2532} or {H430} that we need to replace
def replace_strongs_in_text(text, strongs_mapping):
    """
    Replace Strong's number codes in text with their definitions.
    Converts text to uppercase first for easier reading.
    
    Parameters:
    text: The verse text containing Strong's numbers like {G2532} or {H430}
    strongs_mapping: Dictionary mapping Strong's numbers to their definitions
    
    Returns:
    str: Uppercase text with Strong's numbers replaced by definitions wrapped in *[ ]*
    """
    if pd.isna(text) or text == '':
        return text
    
    # Convert the entire text to uppercase first for easier reading
    text_upper = text.upper()
    
    # Use regex to find all Strong's number patterns: {G####} or {H####}
    # Pattern matches: { followed by G or H, followed by digits, followed by }
    # Note: Since text is now uppercase, the pattern will match {G####} or {H####} in uppercase
    pattern = r'\{([HG]\d+)\}'
    
    def replace_match(match):
        # match.group(1) is the Strong's number without the curly braces (e.g., 'G2532')
        # The match will be uppercase (G or H), but Strong's numbers in the mapping use uppercase anyway
        strongs_num = match.group(1)
        
        # Look up the definition in our mapping dictionary
        if strongs_num in strongs_mapping:
            definition = str(strongs_mapping[strongs_num])
            # Replace {G2532} with the definition text wrapped in *[ ]*
            # Keep definition in original case (not uppercase) for readability
            return f' *[{definition}]* '
        else:
            # If we don't have a definition, keep the original Strong's number
            return match.group(0)  # Returns the full match including braces
    
    # Replace all matches in the uppercase text
    enhanced_text = re.sub(pattern, replace_match, text_upper)
    
    return enhanced_text

# Apply the function to create the text_enhanced column
# Note: Text will be converted to UPPERCASE and definitions will be wrapped in *[ ]* format
if strongs_to_def:
    print("Creating text_enhanced column...")
    print("Note: Text will be converted to UPPERCASE for easier reading")
    print("Note: Strong's definitions will be wrapped in *[ ]* format")
    df_strongs_enhanced['text_enhanced'] = df_strongs_enhanced['text'].apply(
        lambda x: replace_strongs_in_text(x, strongs_to_def)
    )
    print("✓ Created text_enhanced column")
    
    # Show an example
    print("\nExample comparison:")
    print("="*80)
    sample_idx = 0
    print(f"Original text (first 200 chars):")
    print(df_strongs_enhanced.iloc[sample_idx]['text'][:200])
    print(f"\nEnhanced text (first 200 chars):")
    print(df_strongs_enhanced.iloc[sample_idx]['text_enhanced'][:200])
else:
    print("✗ Cannot create text_enhanced column - no Strong's mapping available")


Creating text_enhanced column...
Note: Text will be converted to UPPERCASE for easier reading
Note: Strong's definitions will be wrapped in *[ ]* format
✓ Created text_enhanced column

Example comparison:
Original text (first 200 chars):
In the beginning{H7225} God{H430} created{H1254} the heavens{H8064} and{H853} the earth.{H776}

Enhanced text (first 200 chars):
IN THE BEGINNING *[the first, in place, time, order or rank (specifically, a firstfruit)]*  GOD *[gods in the ordinary sense; but specifically used (in the plural thus, especially with the article) of


In [26]:
# Function to replace Strong's numbers in the strongs column (which is a list) with definitions
# The strongs column contains lists like ['G2532', 'H430', 'G2316']
# We want to replace these with their definitions
def replace_strongs_in_list(strongs_list, strongs_mapping):
    """
    Replace Strong's numbers in a list with their definitions.
    
    Parameters:
    strongs_list: List of Strong's numbers (e.g., ['G2532', 'H430'])
    strongs_mapping: Dictionary mapping Strong's numbers to their definitions
    
    Returns:
    list: List with Strong's numbers replaced by definitions
    """
    if not isinstance(strongs_list, list):
        return strongs_list
    
    # Replace each Strong's number in the list with its definition
    enhanced_list = []
    for strongs_num in strongs_list:
        if strongs_num in strongs_mapping:
            # Replace the number with its definition
            enhanced_list.append(str(strongs_mapping[strongs_num]))
        else:
            # If we don't have a definition, keep the original number
            enhanced_list.append(strongs_num)
    
    return enhanced_list

# Apply the function to create the strongs_enhanced column
if strongs_to_def:
    print("Creating strongs_enhanced column...")
    df_strongs_enhanced['strongs_enhanced'] = df_strongs_enhanced['strongs'].apply(
        lambda x: replace_strongs_in_list(x, strongs_to_def)
    )
    print("✓ Created strongs_enhanced column")
    
    # Show an example
    print("\nExample comparison:")
    print("="*80)
    sample_idx = 0
    print(f"Original strongs (first 10):")
    print(df_strongs_enhanced.iloc[sample_idx]['strongs'][:10])
    print(f"\nEnhanced strongs (first 10):")
    print(df_strongs_enhanced.iloc[sample_idx]['strongs_enhanced'][:10])
else:
    print("✗ Cannot create strongs_enhanced column - no Strong's mapping available")


Creating strongs_enhanced column...
✓ Created strongs_enhanced column

Example comparison:
Original strongs (first 10):
['H7225', 'H430', 'H1254', 'H8064', 'H853', 'H776']

Enhanced strongs (first 10):
['the first, in place, time, order or rank (specifically, a firstfruit)', 'gods in the ordinary sense; but specifically used (in the plural thus, especially with the article) of the supreme God; occasionally applied by way of deference to magistrates; and sometimes as a superlative', '(absolutely) to create; (qualified) to cut down (a wood), select, feed (as formative processes)', 'the sky (as aloft; the dual perhaps alluding to the visible arch in which the clouds move, as well as to the higher ether where the celestial bodies revolve)', 'properly, self (but generally used to point out more definitely the object of a verb or preposition, even or namely)', 'the earth (at large, or partitively a land)']


In [27]:
# Function to remove all Strong's number references from text
# This creates plain text without any Strong's codes like {G2532} or {H430}
def remove_strongs_references(text):
    """
    Remove all Strong's number references from text, leaving only the plain text.
    
    Parameters:
    text: The verse text containing Strong's numbers like {G2532} or {H430}
    
    Returns:
    str: Plain text with all Strong's references removed
    """
    if pd.isna(text) or text == '':
        return text
    
    # Use regex to find and remove all Strong's number patterns: {G####} or {H####}
    # Pattern matches: { followed by G or H, followed by digits, followed by }
    pattern = r'\{[HG]\d+\}'
    
    # Replace all matches with empty string (remove them)
    plain_text = re.sub(pattern, '', text)
    
    # Clean up any extra spaces that might result from removing the references
    # Replace multiple spaces with a single space
    plain_text = re.sub(r'\s+', ' ', plain_text)
    
    # Strip leading and trailing whitespace
    plain_text = plain_text.strip()
    
    return plain_text

# Apply the function to create the plain_text column
print("Creating plain_text column...")
print("This removes all Strong's number references (like {G2532} or {H430}) from the text")
df_strongs_enhanced['plain_text'] = df_strongs_enhanced['text'].apply(remove_strongs_references)
print("✓ Created plain_text column")

# Show an example comparison
print("\nExample comparison:")
print("="*80)
sample_idx = 0
print(f"Original text (first 200 chars):")
print(df_strongs_enhanced.iloc[sample_idx]['text'][:200])
print(f"\nPlain text (first 200 chars):")
print(df_strongs_enhanced.iloc[sample_idx]['plain_text'][:200])


Creating plain_text column...
This removes all Strong's number references (like {G2532} or {H430}) from the text
✓ Created plain_text column

Example comparison:
Original text (first 200 chars):
In the beginning{H7225} God{H430} created{H1254} the heavens{H8064} and{H853} the earth.{H776}

Plain text (first 200 chars):
In the beginning God created the heavens and the earth.


In [28]:
# Display summary information about the enhanced DataFrame
print("Enhanced DataFrame Summary:")
print("="*80)
print(f"Shape: {df_strongs_enhanced.shape} (rows, columns)")
print(f"\nColumns: {list(df_strongs_enhanced.columns)}")
print(f"\nDataFrame Info:")
print(df_strongs_enhanced.info())

# Show a complete example row
print("\n" + "="*80)
print("Complete Example Row:")
print("="*80)
sample_idx = 0
sample_row = df_strongs_enhanced.iloc[sample_idx]
print(f"\nBook: {sample_row['book_name']} {sample_row['chapter']}:{sample_row['verse']}")
print(f"\nOriginal text (first 300 chars):")
print(sample_row['text'][:300])
print(f"\nEnhanced text (first 300 chars):")
print(sample_row['text_enhanced'][:300])
print(f"\nOriginal strongs (first 10): {sample_row['strongs'][:10]}")
print(f"Enhanced strongs (first 10): {sample_row['strongs_enhanced'][:10]}")


Enhanced DataFrame Summary:
Shape: (31086, 10) (rows, columns)

Columns: ['book_name', 'book', 'chapter', 'verse', 'text', 'strongs', 'strongs_count', 'text_enhanced', 'strongs_enhanced', 'plain_text']

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31086 entries, 0 to 31085
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_name         31086 non-null  object
 1   book              31086 non-null  int64 
 2   chapter           31086 non-null  int64 
 3   verse             31086 non-null  int64 
 4   text              31086 non-null  object
 5   strongs           31086 non-null  object
 6   strongs_count     31086 non-null  int64 
 7   text_enhanced     31086 non-null  object
 8   strongs_enhanced  31086 non-null  object
 9   plain_text        31086 non-null  object
dtypes: int64(4), object(6)
memory usage: 2.4+ MB
None

Complete Example Row:

Book: Genesis 1:1

Original text (first 3

In [29]:
# Check how many Strong's numbers were successfully replaced
# Count how many definitions we found vs how many we needed
if strongs_to_def:
    # Count total Strong's numbers in all verses
    total_strongs = sum(len(strongs_list) for strongs_list in df_strongs_enhanced['strongs'])
    
    # Count how many we have definitions for
    strongs_with_defs = sum(
        1 for strongs_list in df_strongs_enhanced['strongs'] 
        for strongs_num in strongs_list 
        if strongs_num in strongs_to_def
    )
    
    coverage = (strongs_with_defs / total_strongs * 100) if total_strongs > 0 else 0
    
    print("Replacement Statistics:")
    print("="*80)
    print(f"Total Strong's numbers in verses: {total_strongs:,}")
    print(f"Strong's numbers with definitions: {strongs_with_defs:,}")
    print(f"Coverage: {coverage:.1f}%")
    
    if coverage < 100:
        # Find which Strong's numbers are missing
        all_strongs = set()
        for strongs_list in df_strongs_enhanced['strongs']:
            all_strongs.update(strongs_list)
        
        missing_strongs = [s for s in all_strongs if s not in strongs_to_def]
        if missing_strongs:
            print(f"\nMissing definitions for {len(missing_strongs)} Strong's numbers:")
            print(f"Sample missing: {sorted(missing_strongs)[:10]}")


Replacement Statistics:
Total Strong's numbers in verses: 348,546
Strong's numbers with definitions: 348,538
Coverage: 100.0%

Missing definitions for 1 Strong's numbers:
Sample missing: ['G0']


## Exporting Enhanced DataFrame to CSV

Save the enhanced DataFrame with Strong's definitions to a CSV file for easy access and sharing.


In [30]:
# Save the enhanced DataFrame to a CSV file
# This will create a file that you can open in Excel, Google Sheets, or any CSV reader
csv_filename = "asvs_strongs_enhanced.csv"

# Export to CSV
# index=False means we don't save the row numbers as a column
# encoding='utf-8' ensures special characters are saved correctly
df_strongs_enhanced.to_csv(csv_filename, index=False, encoding='utf-8')

print(f"✓ Successfully saved enhanced DataFrame to {csv_filename}")
print(f"  - Total rows: {len(df_strongs_enhanced):,}")
print(f"  - Total columns: {len(df_strongs_enhanced.columns)}")
print(f"  - File location: {os.path.abspath(csv_filename)}")

# Show file size
if os.path.exists(csv_filename):
    file_size = os.path.getsize(csv_filename)
    # Convert bytes to MB for readability
    file_size_mb = file_size / (1024 * 1024)
    print(f"  - File size: {file_size_mb:.2f} MB ({file_size:,} bytes)")


✓ Successfully saved enhanced DataFrame to asvs_strongs_enhanced.csv
  - Total rows: 31,086
  - Total columns: 10
  - File location: c:\Users\Development\Documents\GitHub\strongsBibleConcordence\asvs_strongs_enhanced.csv
  - File size: 75.52 MB (79,193,090 bytes)


In [31]:
# Display a preview of what was saved
print("CSV File Preview:")
print("="*80)
print(f"Columns in CSV: {list(df_strongs_enhanced.columns)}")
print(f"\nFirst 3 rows (showing key columns):")
# Show important columns
key_columns = ['book_name', 'chapter', 'verse', 'text', 'text_enhanced']
available_columns = [col for col in key_columns if col in df_strongs_enhanced.columns]
print(df_strongs_enhanced[available_columns].head(3).to_string())


CSV File Preview:
Columns in CSV: ['book_name', 'book', 'chapter', 'verse', 'text', 'strongs', 'strongs_count', 'text_enhanced', 'strongs_enhanced', 'plain_text']

First 3 rows (showing key columns):
  book_name  chapter  verse                                                                                                                                                                                                                            text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## Creating Interactive HTML Bible Study Page

Generate a beautiful, interactive HTML page for conducting Bible study with the enhanced text and Strong's definitions.


In [34]:
df_strongs_enhanced

,book_name,book,chapter,verse,text,strongs,strongs_count,text_enhanced,strongs_enhanced,plain_text
0,Genesis,1,1,1,In the beginning{H7225} God{H430} created{H125...,"[H7225, H430, H1254, H8064, H853, H776]",6,"IN THE BEGINNING *[the first, in place, time, ...","[the first, in place, time, order or rank (spe...",In the beginning God created the heavens and t...
1,Genesis,1,1,2,And the earth{H776} was{H1961} waste{H8414} an...,"[H776, H1961, H8414, H922, H2822, H6440, H8415...",13,"AND THE EARTH *[the earth (at large, or partit...","[the earth (at large, or partitively a land), ...",And the earth was waste and void; and darkness...
2,Genesis,1,1,3,"And God{H430} said,{H559} Let there be{H1961} ...","[H430, H559, H1961, H216, H216]",5,AND GOD *[gods in the ordinary sense; but spec...,[gods in the ordinary sense; but specifically ...,"And God said, Let there be light: and there wa..."
3,Genesis,1,1,4,"And God{H430} saw{H7200} the light,{H216} that...","[H430, H7200, H216, H3588, H2896, H430, H914, ...",10,AND GOD *[gods in the ordinary sense; but spec...,[gods in the ordinary sense; but specifically ...,"And God saw the light, that it was good: and G..."
4,Genesis,1,1,5,And God{H430} called{H7121} the light{H216} Da...,"[H430, H7121, H216, H3117, H2822, H7121, H3915...",11,AND GOD *[gods in the ordinary sense; but spec...,[gods in the ordinary sense; but specifically ...,"And God called the light Day, and the darkness..."
...,...,...,...,...,...,...,...,...,...,...
31081,Revelation,66,22,17,And{G2532} the Spirit{G4151} and{G2532} the br...,"[G2532, G4151, G2532, G3565, G3004, G2064, G25...",19,"AND *[ and, also, even, so then, too, etc.; of...","[ and, also, even, so then, too, etc.; often u...","And the Spirit and the bride say, Come. And he..."
31082,Revelation,66,22,18,{G1063} I testify{G4828} unto every man{G3956}...,"[G1063, G4828, G3956, G191, G3056, G4394, G512...",22,"*[ properly, assigning a reason (used in argu...","[ properly, assigning a reason (used in argume...",I testify unto every man that heareth the word...
31083,Revelation,66,22,19,and{G2532} if{G1437} any man{G5100} shall take...,"[G2532, G1437, G5100, G851, G575, G3056, G976,...",25,"AND *[ and, also, even, so then, too, etc.; of...","[ and, also, even, so then, too, etc.; often u...",and if any man shall take away from the words ...
31084,Revelation,66,22,20,He who{G3588} testifieth{G3140} these things{G...,"[G3588, G3140, G5023, G3004, G3483, G2064, G50...",12,"HE WHO *[ the (sometimes to be supplied, at ot...","[ the (sometimes to be supplied, at others omi...","He who testifieth these things saith, Yea: I c..."


In [32]:
# Convert the enhanced DataFrame to JSON format for use in the HTML page
# This will allow JavaScript to access all the verse data
import json

# Prepare the data for the HTML page
# Convert DataFrame to a list of dictionaries (one per verse)
verses_data = df_strongs_enhanced.to_dict('records')

# Save as JSON file for the HTML page to load
verses_json_filename = "bible_verses_data.json"
with open(verses_json_filename, 'w', encoding='utf-8') as f:
    json.dump(verses_data, f, ensure_ascii=False, indent=2)

print(f"✓ Prepared {len(verses_data)} verses for HTML page")
print(f"✓ Saved verse data to {verses_json_filename}")

# Get unique books for the navigation
unique_books = sorted(df_strongs_enhanced['book_name'].unique())
print(f"\nBooks available: {len(unique_books)}")
print(f"Sample books: {unique_books[:5]}")


✓ Prepared 31086 verses for HTML page
✓ Saved verse data to bible_verses_data.json

Books available: 66
Sample books: ['1 Chronicles', '1 Corinthians', '1 John', '1 Kings', '1 Peter']


In [35]:
# Create the HTML page with embedded JavaScript and CSS
html_content = f'''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Bible Study - Enhanced with Strong's Definitions</title>
    <style>
        * {{
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }}
        
        body {{
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }}
        
        .container {{
            max-width: 1400px;
            margin: 0 auto;
            background: white;
            border-radius: 15px;
            box-shadow: 0 20px 60px rgba(0,0,0,0.3);
            overflow: hidden;
        }}
        
        .header {{
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            text-align: center;
        }}
        
        .header h1 {{
            font-size: 2.5em;
            margin-bottom: 10px;
        }}
        
        .header p {{
            font-size: 1.1em;
            opacity: 0.9;
        }}
        
        .controls {{
            padding: 25px;
            background: #f8f9fa;
            border-bottom: 2px solid #e9ecef;
        }}
        
        .search-section {{
            margin-bottom: 20px;
        }}
        
        .search-section input {{
            width: 100%;
            padding: 12px 20px;
            font-size: 16px;
            border: 2px solid #ddd;
            border-radius: 8px;
            transition: border-color 0.3s;
        }}
        
        .search-section input:focus {{
            outline: none;
            border-color: #667eea;
        }}
        
        .filter-section {{
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-top: 15px;
        }}
        
        .filter-section select {{
            padding: 10px;
            font-size: 14px;
            border: 2px solid #ddd;
            border-radius: 8px;
            background: white;
            cursor: pointer;
        }}
        
        .filter-section select:focus {{
            outline: none;
            border-color: #667eea;
        }}
        
        .view-toggle {{
            display: flex;
            gap: 10px;
            margin-top: 15px;
        }}
        
        .view-toggle button {{
            flex: 1;
            padding: 10px;
            font-size: 14px;
            border: 2px solid #667eea;
            background: white;
            color: #667eea;
            border-radius: 8px;
            cursor: pointer;
            transition: all 0.3s;
        }}
        
        .view-toggle button.active {{
            background: #667eea;
            color: white;
        }}
        
        .view-toggle button:hover {{
            background: #5568d3;
            color: white;
        }}
        
        .content {{
            padding: 30px;
            max-height: 70vh;
            overflow-y: auto;
        }}
        
        .verse-card {{
            background: #f8f9fa;
            border-left: 4px solid #667eea;
            padding: 20px;
            margin-bottom: 20px;
            border-radius: 8px;
            transition: transform 0.2s, box-shadow 0.2s;
        }}
        
        .verse-card:hover {{
            transform: translateX(5px);
            box-shadow: 0 5px 15px rgba(0,0,0,0.1);
        }}
        
        .verse-header {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            margin-bottom: 15px;
            padding-bottom: 10px;
            border-bottom: 2px solid #e9ecef;
        }}
        
        .verse-reference {{
            font-size: 1.3em;
            font-weight: bold;
            color: #667eea;
        }}
        
        .verse-number {{
            color: #6c757d;
            font-size: 0.9em;
        }}
        
        .verse-text {{
            font-size: 1.1em;
            line-height: 1.8;
            color: #333;
            margin-bottom: 15px;
        }}
        
        .verse-text-enhanced {{
            font-size: 1.05em;
            line-height: 2;
            color: #2c3e50;
            background: #fff;
            padding: 15px;
            border-radius: 5px;
            border: 1px solid #e9ecef;
        }}
        
        .strongs-definition {{
            background: #fff3cd;
            border-left: 3px solid #ffc107;
            padding: 8px 12px;
            margin: 5px 0;
            border-radius: 4px;
            font-style: italic;
            color: #856404;
            display: inline-block;
        }}
        
        .stats {{
            display: flex;
            gap: 20px;
            margin-top: 15px;
            font-size: 0.9em;
            color: #6c757d;
        }}
        
        .no-results {{
            text-align: center;
            padding: 60px 20px;
            color: #6c757d;
        }}
        
        .no-results h2 {{
            font-size: 2em;
            margin-bottom: 10px;
        }}
        
        .navigation {{
            display: flex;
            justify-content: space-between;
            align-items: center;
            padding: 20px 30px;
            background: #f8f9fa;
            border-top: 2px solid #e9ecef;
        }}
        
        .nav-button {{
            padding: 12px 24px;
            font-size: 16px;
            border: none;
            background: #667eea;
            color: white;
            border-radius: 8px;
            cursor: pointer;
            transition: background 0.3s;
        }}
        
        .nav-button:hover {{
            background: #5568d3;
        }}
        
        .nav-button:disabled {{
            background: #ccc;
            cursor: not-allowed;
        }}
        
        .verse-counter {{
            color: #6c757d;
            font-size: 14px;
        }}
        
        /* Scrollbar styling */
        .content::-webkit-scrollbar {{
            width: 10px;
        }}
        
        .content::-webkit-scrollbar-track {{
            background: #f1f1f1;
        }}
        
        .content::-webkit-scrollbar-thumb {{
            background: #667eea;
            border-radius: 5px;
        }}
        
        .content::-webkit-scrollbar-thumb:hover {{
            background: #5568d3;
        }}
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>📖 Bible Study</h1>
            <p>Enhanced with Strong's Greek & Hebrew Definitions</p>
        </div>
        
        <div class="controls">
            <div class="search-section">
                <input type="text" id="searchInput" placeholder="Search verses by text, book, or reference...">
            </div>
            
            <div class="filter-section">
                <select id="bookFilter">
                    <option value="">All Books</option>
                    {''.join([f'<option value="{book}">{book}</option>' for book in unique_books])}
                </select>
                
                <select id="chapterFilter">
                    <option value="">All Chapters</option>
                </select>
                
                <select id="verseFilter">
                    <option value="">All Verses</option>
                </select>
            </div>
            
            <div class="view-toggle">
                <button class="active" onclick="setView('enhanced')">Enhanced Text</button>
                <button onclick="setView('original')">Original Text</button>
                <button onclick="setView('both')">Both Views</button>
            </div>
        </div>
        
        <div class="content" id="content">
            <div class="no-results">
                <h2>Loading verses...</h2>
            </div>
        </div>
        
        <div class="navigation">
            <button class="nav-button" id="prevButton" onclick="navigateVerse(-1)">← Previous</button>
            <div class="verse-counter">
                <span id="verseCounter">Verse 1 of {len(verses_data)}</span>
            </div>
            <button class="nav-button" id="nextButton" onclick="navigateVerse(1)">Next →</button>
        </div>
    </div>
    
    <script>
        // Load verse data
        const versesData = {json.dumps(verses_data, ensure_ascii=False)};
        let filteredVerses = [...versesData];
        let currentVerseIndex = 0;
        let currentView = 'enhanced';
        
        // Initialize
        document.addEventListener('DOMContentLoaded', function() {{
            renderVerses();
            setupEventListeners();
        }});
        
        function setupEventListeners() {{
            // Search input
            document.getElementById('searchInput').addEventListener('input', function(e) {{
                filterVerses();
            }});
            
            // Book filter
            document.getElementById('bookFilter').addEventListener('change', function(e) {{
                updateChapterFilter();
                filterVerses();
            }});
            
            // Chapter filter
            document.getElementById('chapterFilter').addEventListener('change', function(e) {{
                updateVerseFilter();
                filterVerses();
            }});
            
            // Verse filter
            document.getElementById('verseFilter').addEventListener('change', function(e) {{
                filterVerses();
            }});
        }}
        
        function updateChapterFilter() {{
            const bookFilter = document.getElementById('bookFilter').value;
            const chapterFilter = document.getElementById('chapterFilter');
            chapterFilter.innerHTML = '<option value="">All Chapters</option>';
            
            if (bookFilter) {{
                const chapters = [...new Set(versesData
                    .filter(v => v.book_name === bookFilter)
                    .map(v => v.chapter)
                    .sort((a, b) => a - b)
                )];
                chapters.forEach(ch => {{
                    chapterFilter.innerHTML += `<option value="${{ch}}">Chapter ${{ch}}</option>`;
                }});
            }}
        }}
        
        function updateVerseFilter() {{
            const bookFilter = document.getElementById('bookFilter').value;
            const chapterFilter = document.getElementById('chapterFilter').value;
            const verseFilter = document.getElementById('verseFilter');
            verseFilter.innerHTML = '<option value="">All Verses</option>';
            
            if (bookFilter && chapterFilter) {{
                const verses = [...new Set(versesData
                    .filter(v => v.book_name === bookFilter && v.chapter == chapterFilter)
                    .map(v => v.verse)
                    .sort((a, b) => a - b)
                )];
                verses.forEach(v => {{
                    verseFilter.innerHTML += `<option value="${{v}}">Verse ${{v}}</option>`;
                }});
            }}
        }}
        
        function filterVerses() {{
            const searchTerm = document.getElementById('searchInput').value.toLowerCase();
            const bookFilter = document.getElementById('bookFilter').value;
            const chapterFilter = document.getElementById('chapterFilter').value;
            const verseFilter = document.getElementById('verseFilter').value;
            
            filteredVerses = versesData.filter(verse => {{
                const matchesSearch = !searchTerm || 
                    verse.text.toLowerCase().includes(searchTerm) ||
                    verse.text_enhanced.toLowerCase().includes(searchTerm) ||
                    `${{verse.book_name}} ${{verse.chapter}}:${{verse.verse}}`.toLowerCase().includes(searchTerm);
                
                const matchesBook = !bookFilter || verse.book_name === bookFilter;
                const matchesChapter = !chapterFilter || verse.chapter == chapterFilter;
                const matchesVerse = !verseFilter || verse.verse == verseFilter;
                
                return matchesSearch && matchesBook && matchesChapter && matchesVerse;
            }});
            
            currentVerseIndex = 0;
            renderVerses();
        }}
        
        function setView(view) {{
            currentView = view;
            document.querySelectorAll('.view-toggle button').forEach(btn => {{
                btn.classList.remove('active');
            }});
            event.target.classList.add('active');
            renderVerses();
        }}
        
        function renderVerses() {{
            const content = document.getElementById('content');
            
            if (filteredVerses.length === 0) {{
                content.innerHTML = `
                    <div class="no-results">
                        <h2>No verses found</h2>
                        <p>Try adjusting your search or filters</p>
                    </div>
                `;
                updateNavigation();
                return;
            }}
            
            const versesToShow = filteredVerses.slice(currentVerseIndex, currentVerseIndex + 10);
            
            content.innerHTML = versesToShow.map(verse => {{
                const reference = `${{verse.book_name}} ${{verse.chapter}}:${{verse.verse}}`;
                let textContent = '';
                
                if (currentView === 'enhanced') {{
                    textContent = `<div class="verse-text-enhanced">${{escapeHtml(verse.text_enhanced)}}</div>`;
                }} else if (currentView === 'original') {{
                    textContent = `<div class="verse-text">${{escapeHtml(verse.text)}}</div>`;
                }} else {{
                    textContent = `
                        <div class="verse-text"><strong>Original:</strong> ${{escapeHtml(verse.text)}}</div>
                        <div class="verse-text-enhanced"><strong>Enhanced:</strong> ${{escapeHtml(verse.text_enhanced)}}</div>
                    `;
                }}
                
                const strongsCount = verse.strongs_count || 0;
                
                return `
                    <div class="verse-card">
                        <div class="verse-header">
                            <div>
                                <span class="verse-reference">${{reference}}</span>
                                <span class="verse-number"> (Book ${{verse.book}}, Verse ${{verse.verse}})</span>
                            </div>
                        </div>
                        ${{textContent}}
                        <div class="stats">
                            <span>Strong's Numbers: ${{strongsCount}}</span>
                        </div>
                    </div>
                `;
            }}).join('');
            
            updateNavigation();
        }}
        
        function escapeHtml(text) {{
            const div = document.createElement('div');
            div.textContent = text;
            return div.innerHTML;
        }}
        
        function updateNavigation() {{
            document.getElementById('verseCounter').textContent = 
                `Showing ${{Math.min(currentVerseIndex + 1, filteredVerses.length)}}-${{Math.min(currentVerseIndex + 10, filteredVerses.length)}} of ${{filteredVerses.length}} verses`;
            
            document.getElementById('prevButton').disabled = currentVerseIndex === 0;
            document.getElementById('nextButton').disabled = currentVerseIndex + 10 >= filteredVerses.length;
        }}
        
        function navigateVerse(direction) {{
            const step = 10;
            const newIndex = currentVerseIndex + (direction * step);
            
            if (newIndex >= 0 && newIndex < filteredVerses.length) {{
                currentVerseIndex = newIndex;
                renderVerses();
                document.getElementById('content').scrollTop = 0;
            }}
        }}
    </script>
</body>
</html>
'''

# Save the HTML file
html_filename = "bible_study.html"
with open(html_filename, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"✓ Created interactive Bible study HTML page: {html_filename}")
print(f"  - Total verses: {len(verses_data):,}")
print(f"  - File location: {os.path.abspath(html_filename)}")
print(f"\nOpen {html_filename} in your web browser to start studying!")


✓ Created interactive Bible study HTML page: bible_study.html
  - Total verses: 31,086
  - File location: c:\Users\Development\Documents\GitHub\strongsBibleConcordence\bible_study.html

Open bible_study.html in your web browser to start studying!
